##  Get input  real estate listings from Team Wilson site

## Imports

In [1]:
from bs4 import BeautifulSoup
import requests
from argparse import Namespace
import re
import pandas as pd
from datetime import date
import os


## Settings

In [2]:
args = Namespace(
output_dir = 'C:\gitrepos\github\MLS\data',
results_url = 'https://www.teamwilsonrealestate.com/search/results',
details_url = 'https://www.teamwilsonrealestate.com/search/details',
chandler_suffix = '/1jg/',
phx_suffix = '/1jh/',
tempe_suffix = '/1ji/'

)

## Main Page Cookies and Headers

In [3]:
cookies = {
    'csrftoken': 'cEQCyTwkev79QN0qLsa8j2IK6MXaRrVrgQ32mJ2pau2PgKlYT6f7sdqw0sZPefVY',
    '_fbp': 'fb.1.1647927256049.1535127417',
    '__utma': '150816819.713123125.1647927256.1647927256.1647927256.1',
    '__utmc': '150816819',
    '__utmz': '150816819.1647927256.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none)',
    '__utmt_UA-50697213-2': '1',
    '__utmt': '1',
    'last_visit_time': '1647927256',
    '__utmb': '150816819.4.10.1647927256',
    'member_uuid': '7a855150-a34a-11ec-b871-0242ac110003',
}

headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://www.teamwilsonrealestate.com/member/saved_searches/',
    'Accept-Language': 'en-US,en;q=0.9',
    # Requests sorts cookies= alphabetically
    # 'Cookie': 'csrftoken=cEQCyTwkev79QN0qLsa8j2IK6MXaRrVrgQ32mJ2pau2PgKlYT6f7sdqw0sZPefVY; _fbp=fb.1.1647927256049.1535127417; __utma=150816819.713123125.1647927256.1647927256.1647927256.1; __utmc=150816819; __utmz=150816819.1647927256.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utmt_UA-50697213-2=1; __utmt=1; last_visit_time=1647927256; __utmb=150816819.4.10.1647927256; member_uuid=7a855150-a34a-11ec-b871-0242ac110003',
}

response = requests.get('https://www.teamwilsonrealestate.com/member/saved_searches/', headers=headers, cookies=cookies)

In [4]:
with open('main.html', 'w') as outfile:
    outfile.write(str(response.content))

In [5]:
def get_listings(url):

    response = requests.get(url, headers=headers, cookies=cookies)
    soup = BeautifulSoup(response.content, 'lxml')

    property_dict = {'address':[],'price':[], 'baths':[],  'beds':[], 'MLS':[], 'sqft':[]}
    property_results = soup.find_all('div', class_ = 'row property results')
    for property_result in property_results:
        address = property_result.find('a', class_='address').text.strip()
        price = property_result.find('a', class_='price').text
        price = ''.join(price.split())

        # Beds, Baths, MLS and SQFT
        property_section_detail = property_result.find('ul')
        property_attribs = []
        items = property_section_detail.find_all('span', class_='number')
        for item in items:
            property_attribs.append(item.text)
        beds, baths, MLS, sqft = property_attribs


        property_dict['address'].append(address)
        property_dict['price'].append(price)
        property_dict['beds'].append(beds)
        property_dict['baths'].append(baths)
        property_dict['MLS'].append(MLS)
        property_dict['sqft'].append(sqft)

    return pd.DataFrame(property_dict)


def number_listings(soup):
    total_properties = soup.find('div', class_ = 'show-for-medium-up properties-found').text
    #  expected string "20 properties" , extract the number 20
    pattern = r'([0-9]+)\s'
    total_properties = int(re.findall(pattern=pattern, string=total_properties)[0])
    return total_properties


### Chandler

In [6]:
url = args.results_url + args.chandler_suffix
df = get_listings(url)
df.set_index(['MLS'], inplace=True)
chandler_listings  = df
chandler_listings

,address,price,baths,beds,sqft
MLS,,,,,
6392889,"1610 E Whitten Street, Chandler","$650,000",2,4,"2,325"
6382398,"2054 E Indian Wells Drive, Chandler","$650,000",2,4,"2,010"
6380880,"5940 S White Place, Chandler","$650,000",2,4,"2,511"
6386642,"390 N Corsica Place, Chandler","$649,900▼",2,4,"2,182"
6311079,"4571 W Toledo Street, Chandler","$642,000▼",3,5,"2,908"
6389984,"2841 W Gail Drive, Chandler","$629,900",2.5,4,"2,267"
6384936,"1442 E Saragosa Street, Chandler","$629,000▼",2.5,3,"2,154"
6395249,"1753 E Boston Circle, Chandler","$625,000",2.5,4,"2,086"
6394917,"2170 E Augusta Avenue, Chandler","$625,000",2,3,"2,159"


In [7]:
num_listing_chandler = chandler_listings.shape[0]

### Phoenix

In [8]:
url = args.results_url + args.phx_suffix
df = get_listings(url)
df.set_index(['MLS'], inplace=True)
phx_listings  = df
phx_listings

,address,price,baths,beds,sqft
MLS,,,,,
6388258,"3414 E Woodland Drive, Phoenix","$650,000",2.5,4,"2,316"
6377253,"5219 W Desert Hollow Drive, Phoenix","$650,000",2,4,"1,998"
6391772,"2708 W Patagonia Way, Phoenix","$649,900",2.5,4,"1,965"
6389704,"22824 N 21st Street, Phoenix","$645,000▲",2,3,"2,050"
6383456,"3204 W Molly Lane, Phoenix","$640,000",2.5,4,"2,874"
6391108,"19801 N 15th Place, Phoenix","$639,900",2,3,"1,883"
6383537,"3906 E Amberwood Drive, Phoenix","$630,000▲",2,3,"1,899"
6370353,"4410 E Dry Creek Road, Phoenix","$629,000",2.5,3,"2,124"
6388264,"1718 W Satinwood Drive, Phoenix","$625,000",3,4,"1,937"


In [9]:
num_listing_phx = phx_listings.shape[0]
print(num_listing_phx)

30


### Tempe

In [10]:
url = args.results_url + args.tempe_suffix
df = get_listings(url)
df.set_index(['MLS'], inplace=True)
tempe_listings  = df
tempe_listings

,address,price,baths,beds,sqft
MLS,,,,,
6395287,"2134 E Watson Drive, Tempe","$650,000",2,4,"1,800"
6384990,"2404 E Geneva Drive, Tempe","$625,000",2,4,"2,241"
6393533,"129 E Fremont Drive, Tempe","$600,000",2,3,"1,880"
6391384,"1302 E Ellis Drive, Tempe","$599,000▼",2,3,"1,852"
6369170,"1924 E Loyola Drive, Tempe","$530,072▲",2,3,"1,762"


In [11]:
num_listing_tempe = tempe_listings.shape[0]

## Combined Listing Details

In [12]:
frames = [chandler_listings, phx_listings, tempe_listings]
combined_listings = pd.concat(frames)
combined_listings

,address,price,baths,beds,sqft
MLS,,,,,
6392889,"1610 E Whitten Street, Chandler","$650,000",2,4,"2,325"
6382398,"2054 E Indian Wells Drive, Chandler","$650,000",2,4,"2,010"
6380880,"5940 S White Place, Chandler","$650,000",2,4,"2,511"
6386642,"390 N Corsica Place, Chandler","$649,900▼",2,4,"2,182"
6311079,"4571 W Toledo Street, Chandler","$642,000▼",3,5,"2,908"
6389984,"2841 W Gail Drive, Chandler","$629,900",2.5,4,"2,267"
6384936,"1442 E Saragosa Street, Chandler","$629,000▼",2.5,3,"2,154"
6395249,"1753 E Boston Circle, Chandler","$625,000",2.5,4,"2,086"
6394917,"2170 E Augusta Avenue, Chandler","$625,000",2,3,"2,159"


## Details

In [13]:
cookies = {
    'assigned-user-83148': '%7B%22id%22%3A83148%2C%22first_name%22%3A%22Jeremy%22%2C%22last_name%22%3A%22Wilson%22%2C%22email%22%3A%22Jeremy%40teamwilsonrealestate.com%22%2C%22role%22%3A%22Agent%22%2C%22avatar%22%3A%22https%3A%2F%2Fam1-rg-lead-manager-profile.s3.amazonaws.com%2F91bded48a2444f90a2d0194adebb0a28-CombinedHeadshotColorPNG.png%22%7D',
    '__utmz': '150816819.1647884633.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none)',
    'last_content_search_id': '2',
    'csrftoken': 'E03rxMqEqQOy86CrnMEAta2QwCgncLUU9aps5Iu7q81jT6iIMZyd8EHneSjySpue',
    'member_uuid': '7a855150-a34a-11ec-b871-0242ac110003',
    'first_search_id': '1988',
    '__utmc': '150816819',
    '__utma': '150816819.1341560933.1647884633.1647926856.1648082070.4',
    '__utmb': '150816819.46.10.1648082070',
    'last_visit_time': '1648083195',
}

headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://www.teamwilsonrealestate.com/search/results/1jg/',
    'Accept-Language': 'en-US,en;q=0.9',
    # Requests sorts cookies= alphabetically
    # 'Cookie': 'assigned-user-83148=%7B%22id%22%3A83148%2C%22first_name%22%3A%22Jeremy%22%2C%22last_name%22%3A%22Wilson%22%2C%22email%22%3A%22Jeremy%40teamwilsonrealestate.com%22%2C%22role%22%3A%22Agent%22%2C%22avatar%22%3A%22https%3A%2F%2Fam1-rg-lead-manager-profile.s3.amazonaws.com%2F91bded48a2444f90a2d0194adebb0a28-CombinedHeadshotColorPNG.png%22%7D; __utmz=150816819.1647884633.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); last_content_search_id=2; csrftoken=E03rxMqEqQOy86CrnMEAta2QwCgncLUU9aps5Iu7q81jT6iIMZyd8EHneSjySpue; member_uuid=7a855150-a34a-11ec-b871-0242ac110003; first_search_id=1988; __utmc=150816819; __utma=150816819.1341560933.1647884633.1647926856.1648082070.4; __utmb=150816819.46.10.1648082070; last_visit_time=1648083195',
}


In [14]:
def get_listing_details(url):
    '''
    Input the detailed url for each property.  Updates the prop_detail_dict with the details of the property
    '''
 
    response = requests.get(url, headers=headers, cookies=cookies)
    soup = BeautifulSoup(response.content, 'lxml')

    details = []
    results_details = soup.find('div', class_ = 'results-details')
    prop_details = results_details.find_all('div', class_='prop-descrip property_detail_specs')
    for prop_detail in prop_details:
        lst = prop_detail.text.split('\n')
        lst = [i for i in lst if i]
        if lst[0] in prop_detail_dict:
            prop_detail_dict[lst[0]].append(lst[1])
        else:
            print(url)
            print(f'{lst[0]} does not exit index')


In [15]:
prop_detail_dict = {'Days on Market': [],
 'Status': [],
 'MLS#': [],
 'City': [],
 'Bedrooms': [],
 'Bathrooms': [],
 'Living SQFT': [],
 'Lot Size': [],
 'Subdivision': [],
 'Year Built': [],
 'Type': []}

## Chandler

In [16]:
for i in range(0,num_listing_chandler): 
    url = args.details_url + args.chandler_suffix +  str(i)
    get_listing_details(url)



https://www.teamwilsonrealestate.com/search/details/1jg/3
Price Change does not exit index
https://www.teamwilsonrealestate.com/search/details/1jg/4
Price Change does not exit index
https://www.teamwilsonrealestate.com/search/details/1jg/6
Price Change does not exit index
https://www.teamwilsonrealestate.com/search/details/1jg/14
Price Change does not exit index
https://www.teamwilsonrealestate.com/search/details/1jg/19
Price Change does not exit index
https://www.teamwilsonrealestate.com/search/details/1jg/20
Price Change does not exit index


## Phoenix

In [17]:
for i in range(0,num_listing_phx): 
    url = args.details_url + args.phx_suffix +  str(i)
    get_listing_details(url)


https://www.teamwilsonrealestate.com/search/details/1jh/3
Price Change does not exit index
https://www.teamwilsonrealestate.com/search/details/1jh/6
Price Change does not exit index
https://www.teamwilsonrealestate.com/search/details/1jh/11
Price Change does not exit index
https://www.teamwilsonrealestate.com/search/details/1jh/23
Price Change does not exit index


## Tempe

In [18]:
for i in range(0,num_listing_tempe): 
    url = args.details_url + args.tempe_suffix +  str(i)
    get_listing_details(url)

https://www.teamwilsonrealestate.com/search/details/1ji/3
Price Change does not exit index
https://www.teamwilsonrealestate.com/search/details/1ji/4
Price Change does not exit index


In [19]:
df = pd.DataFrame(prop_detail_dict)
df = df.rename(columns={'MLS#':'MLS'})
df.set_index(['MLS'], inplace=True)
combined_property_details = df
combined_property_details


,Days on Market,Status,City,Bedrooms,Bathrooms,Living SQFT,Lot Size,Subdivision,Year Built,Type
MLS,,,,,,,,,,
6392889,4,ACTIVE,Chandler,4,2,"2,325","9,766",Chandler Ranch Unit 1,1988,Single Family - Detached
6382398,25,ACTIVE,Chandler,4,2,"2,010","7,020",Cooper Commons,2001,Single Family - Detached
6380880,29,UCB (UNDER CONTRACT-BACKUPS),Chandler,4,2,"2,511","8,065",Shadow Ridge,2004,Single Family - Detached
6386642,16,ACTIVE,Chandler,4,2,"2,182","7,719",Sunset Cove,1996,Single Family - Detached
6311079,197,UCB (UNDER CONTRACT-BACKUPS),Chandler,5,3,"2,908","4,957",Wild Tree,1995,Single Family - Detached
6389984,10,ACTIVE,Chandler,4,2.5,"2,267","5,750",Autumn Lane,1994,Single Family - Detached
6384936,20,ACTIVE,Chandler,3,2.5,"2,154","7,200",Willis Ranch Unit 1,2000,Single Family - Detached
6395249,1,ACTIVE,Chandler,4,2.5,"2,086","4,931",Springs Unit 2 Amd,1994,Single Family - Detached
6394917,1,ACTIVE,Chandler,3,2,"2,159","10,681",Cooper Commons Parcel 1,1999,Single Family - Detached


## Merge both listings and details

In [20]:
frames = [combined_listings, combined_property_details]
homes = pd.concat(frames, axis=1)
homes['Days on Market'] = homes['Days on Market'].astype(int)
homes_daily_status = homes[[ 'price', 'Days on Market', 'Status']]
home_property_details= homes[['address', 'baths', 'beds', 'sqft', 'City', 'Bedrooms', 'Bathrooms', 
                        'Living SQFT', 'Lot Size','Subdivision', 'Year Built', 'Type']]

## ACTIVE Listings

In [21]:
# Active Homes
active_homes = homes[homes.Status=='ACTIVE'].sort_values(by=['Days on Market'])
active_homes

,address,price,baths,beds,sqft,Days on Market,Status,City,Bedrooms,Bathrooms,Living SQFT,Lot Size,Subdivision,Year Built,Type
MLS,,,,,,,,,,,,,,,
6395832,"2557 W Bisbee Way, Phoenix","$550,000",2,4,"1,827",0,ACTIVE,Phoenix,4,2,"1,827","5,209",Anthem Unit 43,2003,Single Family - Detached
6395287,"2134 E Watson Drive, Tempe","$650,000",2,4,"1,800",1,ACTIVE,Tempe,4,2,"1,800","7,294",Knoell Tempe,1978,Single Family - Detached
6395249,"1753 E Boston Circle, Chandler","$625,000",2.5,4,"2,086",1,ACTIVE,Chandler,4,2.5,"2,086","4,931",Springs Unit 2 Amd,1994,Single Family - Detached
6394917,"2170 E Augusta Avenue, Chandler","$625,000",2,3,"2,159",1,ACTIVE,Chandler,3,2,"2,159","10,681",Cooper Commons Parcel 1,1999,Single Family - Detached
6395027,"1293 W Cindy Street, Chandler","$589,000",2,4,"1,809",1,ACTIVE,Chandler,4,2,"1,809","13,591",Rialto Hills Two Lot 95-193 Tr A-D,1991,Single Family - Detached
6395341,"910 E Blackhawk Drive, Phoenix","$596,000",2,4,"1,810",1,ACTIVE,Phoenix,4,2,"1,810","5,310",Eagle Summit Amd,1999,Single Family - Detached
6393868,"1380 W Pelican Court, Chandler","$580,000",3,4,"2,053",2,ACTIVE,Chandler,4,3,"2,053","7,612",Clemente Ranch,1998,Single Family - Detached
6393533,"129 E Fremont Drive, Tempe","$600,000",2,3,"1,880",3,ACTIVE,Tempe,3,2,"1,880","9,635",Tempe Gardens 3 Lot 180-320 Tr F,1964,Single Family - Detached
6393624,"3341 E Nighthawk Way, Phoenix","$550,000",3,4,"2,299",3,ACTIVE,Phoenix,4,3,"2,299","5,493",Lakewood Parcel 18 Lot 1-134 Tr A-E,1988,Single Family - Detached


In [22]:
activeMLS = list(active_homes.index)
searchurl = 'https://www.teamwilsonrealestate.com/search/results/?mls_number=' + "+".join(activeMLS)
print(searchurl)

https://www.teamwilsonrealestate.com/search/results/?mls_number=6395832+6395287+6395249+6394917+6395027+6395341+6393868+6393533+6393624+6392889+6392160+6391772+6391384+6390832+6390346+6390715+6389704+6389984+6388258+6388074+6388089+6388264+6386642+6384936+6383456+6382130+6382398+6377253+6369170+6366593


https://www.teamwilsonrealestate.com/search/results/?mls_number=6375212+6375333

## Under Contract Listings

In [23]:
homes[homes.Status!='ACTIVE'].sort_values(by=['Days on Market'])

,address,price,baths,beds,sqft,Days on Market,Status,City,Bedrooms,Bathrooms,Living SQFT,Lot Size,Subdivision,Year Built,Type
MLS,,,,,,,,,,,,,,,
6394134,"4108 E Tanglewood Drive, Phoenix","$565,000",2,4,"2,001",2,UCB (UNDER CONTRACT-BACKUPS),Phoenix,4,2,"2,001","6,547",Remington Trace Lot 110-217 Tr A-F,1988,Single Family - Detached
6391108,"19801 N 15th Place, Phoenix","$639,900",2,3,"1,883",8,UCB (UNDER CONTRACT-BACKUPS),Phoenix,3,2,"1,883","6,430",Eagle Hills,1997,Single Family - Detached
6389896,"3131 W Covey Lane, Phoenix","$595,000",2.5,4,"2,440",10,UCB (UNDER CONTRACT-BACKUPS),Phoenix,4,2.5,"2,440","6,489",Foothills North 3,1999,Single Family - Detached
6387499,"4636 E Bighorn Avenue, Phoenix","$610,000",2,4,"1,873",15,UCB (UNDER CONTRACT-BACKUPS),Phoenix,4,2,"1,873","6,800",Desert Breeze Lot 1-188 Tr A & Drainageway,1993,Single Family - Detached
6386223,"1098 N Longmore Street, Chandler","$549,000▼",2.5,3,"1,786",17,UCB (UNDER CONTRACT-BACKUPS),Chandler,3,2.5,"1,786","5,388",Hamilton Homes 2 At Andersen Springs 1-125 Tr A-J,1991,Single Family - Detached
6384990,"2404 E Geneva Drive, Tempe","$625,000",2,4,"2,241",20,UCB (UNDER CONTRACT-BACKUPS),Tempe,4,2,"2,241","9,261",Tempe Royal Palms 8,1970,Single Family - Detached
6384231,"2170 E Iris Drive, Chandler","$625,000",2,3,"2,127",22,UCB (UNDER CONTRACT-BACKUPS),Chandler,3,2,"2,127","7,475",Redwood Estates,2004,Single Family - Detached
6383678,"6261 S Pearl Drive, Chandler","$590,000▲",2.5,3,"1,906",22,UCB (UNDER CONTRACT-BACKUPS),Chandler,3,2.5,"1,906","5,456",Sun Groves,2006,Single Family - Detached
6383785,"1764 W Brookwood Court, Phoenix","$599,000",3,3,"2,180",22,UCB (UNDER CONTRACT-BACKUPS),Phoenix,3,3,"2,180","5,275",Foothills Club West Parcel 17,2000,Single Family - Detached


In [24]:
homes_daily_status['date'] = str(date.today())

<ipython-input-24-3cfc1675a111>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  homes_daily_status['date'] = str(date.today())


In [25]:
homes_daily_status

,price,Days on Market,Status,date
MLS,,,,
6392889,"$650,000",4,ACTIVE,2022-05-07
6382398,"$650,000",25,ACTIVE,2022-05-07
6380880,"$650,000",29,UCB (UNDER CONTRACT-BACKUPS),2022-05-07
6386642,"$649,900▼",16,ACTIVE,2022-05-07
6311079,"$642,000▼",197,UCB (UNDER CONTRACT-BACKUPS),2022-05-07
6389984,"$629,900",10,ACTIVE,2022-05-07
6384936,"$629,000▼",20,ACTIVE,2022-05-07
6395249,"$625,000",1,ACTIVE,2022-05-07
6394917,"$625,000",1,ACTIVE,2022-05-07


In [26]:
home_property_details

,address,baths,beds,sqft,City,Bedrooms,Bathrooms,Living SQFT,Lot Size,Subdivision,Year Built,Type
MLS,,,,,,,,,,,,
6392889,"1610 E Whitten Street, Chandler",2,4,"2,325",Chandler,4,2,"2,325","9,766",Chandler Ranch Unit 1,1988,Single Family - Detached
6382398,"2054 E Indian Wells Drive, Chandler",2,4,"2,010",Chandler,4,2,"2,010","7,020",Cooper Commons,2001,Single Family - Detached
6380880,"5940 S White Place, Chandler",2,4,"2,511",Chandler,4,2,"2,511","8,065",Shadow Ridge,2004,Single Family - Detached
6386642,"390 N Corsica Place, Chandler",2,4,"2,182",Chandler,4,2,"2,182","7,719",Sunset Cove,1996,Single Family - Detached
6311079,"4571 W Toledo Street, Chandler",3,5,"2,908",Chandler,5,3,"2,908","4,957",Wild Tree,1995,Single Family - Detached
6389984,"2841 W Gail Drive, Chandler",2.5,4,"2,267",Chandler,4,2.5,"2,267","5,750",Autumn Lane,1994,Single Family - Detached
6384936,"1442 E Saragosa Street, Chandler",2.5,3,"2,154",Chandler,3,2.5,"2,154","7,200",Willis Ranch Unit 1,2000,Single Family - Detached
6395249,"1753 E Boston Circle, Chandler",2.5,4,"2,086",Chandler,4,2.5,"2,086","4,931",Springs Unit 2 Amd,1994,Single Family - Detached
6394917,"2170 E Augusta Avenue, Chandler",2,3,"2,159",Chandler,3,2,"2,159","10,681",Cooper Commons Parcel 1,1999,Single Family - Detached


## Serialize the files

In [27]:
filename = os.path.join(args.output_dir, 'homes_daily_status_' + str(date.today()) + '.tsv') 
homes_daily_status.to_csv(filename, sep='\t')

In [28]:
filename = os.path.join(args.output_dir, 'home_property_details_' + str(date.today()) + '.tsv') 
home_property_details.to_csv(filename, sep='\t')